# Goal

# Project Plan

- acquire at least 

In [ ]:


Predict missing values from the data set using clustering
Derive insights on "Armstrong", the person from whom the fitness data was generated
Hypotheses
There are patterns on Armstrong's daily activities.
Armstrong's fitness activity is affected by the month/season of the year.
Data Science Skills Used
Data prep and cleaning
Time Series Analysis
Visualization
Modeling using Parametric Methods
Modeling using the Prophet algorithm
Evaluation of Models

# Data Dictionary

## Summary of Data

In [84]:
# imports
import unicodedata
import re
import json

import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

# matplotlib default plotting styles
plt.rc("patch", edgecolor="black", force_edgecolor=True)
plt.rc("axes", grid=True)
plt.rc("grid", linestyle=":", linewidth=0.8, alpha=0.7)
plt.rc("axes.spines", right=False, top=False)
plt.rc("figure", figsize=(11, 8))
plt.rc("font", size=12.0)
plt.rc("hist", bins=25)

import acquire

SyntaxError: invalid syntax (acquire.py, line 23)

In [5]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
import requests
from bs4 import BeautifulSoup as bs
import re
import pandas as pd
import os

def acquire(html = 'https://github.com/trending?since=weekly'):
    url = requests.get(html)
    return bs(url.content, 'html.parser')
def webcrawler_repos(s):
    index = {'language' : [], 'link' : []}
    box_row = s.find_all('article')
    for soup in box_row:
        title = soup.findAll("span", itemprop="programmingLanguage")
        if len(title) == 0:
            continue
        for t in title:
            index['language'].append(t.get_text(strip = True))
        a = soup.find_all('a', href = True)
        a = [str(aa) for aa in a if 'text-normal' in str(aa) and \
         """lh-condensed-ultra d-block""" not in str(aa)]
        for aa in a:
            try:
                aa = re.sub('"', '', aa[aa.index('=') + 1:aa.index('>')])
                index['link'].append(aa)
            except:
                continue
    return pd.DataFrame(index)
def cleanhtml(raw_html):
    cleanr = re.compile(r'<.*?>')
    cleantext = re.sub(cleanr, '', r'%s' % raw_html)
    return cleantext
def webscrape_readme(data):
    index = {'link' : [], 'readme' : []}
    for l in data['link']:
        index['link'].append(l)
        try:
            url = requests.get('https://www.github.com' + l + \
                               '/blob/master/README.md')
        except:
            print('ERROR')
        soup = bs(url.content, 'html.parser')
        sp = soup.find_all('p')
        html_list = []
        for s in sp:
            html_list.append(cleanhtml(s))
        index['readme'].append(html_list)
    return pd.DataFrame(index)


In [21]:
def get_github():
    data = webcrawler_repos(acquire())
    subset = webscrape_readme(data)
    data = data.merge(subset, on = 'link')
    data['target'] = data['readme'].apply(lambda x: x[1:])
    data.rename(columns = {'language':'label'}, inplace = True)
    data.drop(columns = ['link', 'readme'], inplace = True)
    return data

In [22]:
def store_github_data():
    filename = './github.csv'

    # check for presence of the file or make a new request
    if os.path.exists(filename):
        return pd.read_csv(filename)
    else:
        return get_github()

In [97]:
df = store_github_data()

In [98]:
df.shape

(25, 2)

In [25]:
df.head()

,label,target
0,Python,[We provide an implementation of Demucs and Co...
1,JavaScript,[由于部分内容被控评和删帖，已经无法在中文互联网搜索到，所以本项目将收集华为作过的恶，记录这...
2,HTML,[]
3,Java,"[English| 简体中文, Alink是基于Flink的通用算法平台,由阿里巴巴计算平台..."
4,JavaScript,"[\n, This repository contains JavaScript based..."


In [94]:
def normalize(string):
    """
    Convert to all lowercase  
    Normalize the unicode chars  
    Remove any non-alpha or whitespace characters  
    Remove any alpha strings with 2 characters or less  
    """
    string = string.lower()
    string = unicodedata.normalize('NFKD', string).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    
    # keep only alpha chars
    string = re.sub(r'[^a-z]', ' ', string)
    
    # remove strings less than 2 chars in length
    string = re.sub(r'\b[a-z]{,2}\b', '', string)
    
    # convert newlines and tabs to a single space
    string = re.sub(r'[\r|\n|\r\n]+', ' ', string)
    
    # strip extra whitespace
    string = string.strip()
    
    return string

def stem(string):
    ps = nltk.porter.PorterStemmer()
    stems = [ps.stem(word) for word in string.split()]
    string_of_stems = ' '.join(stems)
    return string_of_stems

def lemmatize(string):
    wnl = nltk.stem.WordNetLemmatizer()
    lemmas = [wnl.lemmatize(word) for word in string.split()]
    string_of_lemmas = ' '.join(lemmas)
    return string_of_lemmas

def tokenize(string):
    tokenizer = nltk.tokenize.ToktokTokenizer()
    return tokenizer.tokenize(string, return_str=True)

def remove_stopwords(tokenized_string, extra_words=[], exclude_words=[]):
    words = tokenized_string.split()
    stopword_list = stopwords.words('english')

    # remove the excluded words from the stopword list
    stopword_list = set(stopword_list) - set(exclude_words)

    # add in the user specified extra words
    stopword_list = stopword_list.union(set(extra_words))

    filtered_words = [w for w in words if w not in stopword_list]
    final_string = " ".join(filtered_words)
    return final_string

def prep_target(df, col):
    df[col] = df[col].apply(', '.join)
    df = df.assign(original = df[col])
    #df = df['original'].apply(', '.join)
    df = df.assign(normalized = df.original.apply(normalize))
    df = df.assign(stemmed = df.normalized.apply(stem))
    df = df.assign(lemmatized = df.normalized.apply(lemmatize))
    df = df.assign(cleaned = df.lemmatized.apply(remove_stopwords))
    #df.drop(columns=["title"], inplace=True)
    return df

In [99]:
cleaned_df = prep_target(df, 'target')

In [100]:
cleaned_df

,label,target,original,normalized,stemmed,lemmatized,cleaned
0,Python,We provide an implementation of Demucs and Con...,We provide an implementation of Demucs and Con...,provide implementation demucs and conv tasne...,provid implement demuc and conv tasnet for mus...,provide implementation demucs and conv tasnet ...,provide implementation demucs conv tasnet musi...
1,JavaScript,由于部分内容被控评和删帖，已经无法在中文互联网搜索到，所以本项目将收集华为作过的恶，记录这些...,由于部分内容被控评和删帖，已经无法在中文互联网搜索到，所以本项目将收集华为作过的恶，记录这些...,,,,
2,HTML,,,,,,
3,Java,"English| 简体中文, Alink是基于Flink的通用算法平台,由阿里巴巴计算平台P...","English| 简体中文, Alink是基于Flink的通用算法平台,由阿里巴巴计算平台P...",english alinkflink pai alink jupyter note...,english alinkflink pai alink jupyt notebook py...,english alinkflink pai alink jupyter notebook ...,english alinkflink pai alink jupyter notebook ...
4,JavaScript,"\n, This repository contains JavaScript based ...","\n, This repository contains JavaScript based ...",this repository contains javascript based exam...,thi repositori contain javascript base exampl ...,this repository contains javascript based exam...,repository contains javascript based example m...
5,C++,,,,,,
6,JavaScript,Collection of 8000+ publicly available IPTV ch...,Collection of 8000+ publicly available IPTV ch...,collection publicly available iptv chan...,collect publicli avail iptv channel from all o...,collection publicly available iptv channel fro...,collection publicly available iptv channel wor...
7,TypeScript,A secure runtime for JavaScript and TypeScript...,A secure runtime for JavaScript and TypeScript...,secure runtime for javascript and typescript ...,secur runtim for javascript and typescript den...,secure runtime for javascript and typescript d...,secure runtime javascript typescript deno aim ...
8,Java,", 当前最新版本： 2.1.2（发布日期：20191122）, \n\n\n\n, Jeec...",", 当前最新版本： 2.1.2（发布日期：20191122）, \n\n\n\n, Jeec...",jeecgboot springboot ant design amp vue m...,jeecgboot springboot ant design amp vue mybati...,jeecgboot springboot ant design amp vue mybati...,jeecgboot springboot ant design amp vue mybati...
9,Java,"Halo 是一款现代化的个人独立博客系统，给习惯写博客的同学多一个选择。, \n\n\n\n...","Halo 是一款现代化的个人独立博客系统，给习惯写博客的同学多一个选择。, \n\n\n\n...",halo halo helo java ...,halo halo helo java telegram telegram wehalo h...,halo halo helo java telegram telegram wehalo h...,halo halo helo java telegram telegram wehalo h...


In [122]:
df = pd.read_csv('hola.csv')
df.head()

,Unnamed: 0,index,language,link,readme
0,0,0,JavaScript,freeCodeCamp/freeCodeCamp,['GitHub is home to over 40 million developers...
1,1,1,Rust,996icu/996.ICU,['GitHub is home to over 40 million developers...
2,2,2,JavaScript,vuejs/vue,['GitHub is home to over 40 million developers...
3,3,3,JavaScript,facebook/react,['GitHub is home to over 40 million developers...
4,4,4,C++,tensorflow/tensorflow,['GitHub is home to over 40 million developers...


In [123]:
def basic_clean(data):
    data['target'] = data['readme'].apply(lambda x: x[1:])
    data.rename(columns = {'language':'label'}, inplace = True)
    data.drop(columns = ['link', 'readme','Unnamed: 0','index'], inplace = True)
    return data

In [125]:
df = basic_clean(df)

In [126]:
df

,label,target
0,JavaScript,'GitHub is home to over 40 million developers ...
1,Rust,'GitHub is home to over 40 million developers ...
2,JavaScript,'GitHub is home to over 40 million developers ...
3,JavaScript,'GitHub is home to over 40 million developers ...
4,C++,'GitHub is home to over 40 million developers ...
5,JavaScript,'GitHub is home to over 40 million developers ...
6,Shell,'GitHub is home to over 40 million developers ...
7,JavaScript,'\n GitHub makes it easy to scale bac...
8,Rust,'GitHub is home to over 40 million developers ...
9,JavaScript,'GitHub is home to over 40 million developers ...
